In [5]:
from decode_util import Decode_util

In [6]:
with open('/home/yuanzn/Documents/NSF_cohort/code/h264_structure_decode/foreman_cif.h264', 'rb') as h264_file:
    content = h264_file.read()
    print(type(content))
    print(str(content[0:3]))
    str_content = ''.join(format(byte, '02x') for byte in content)
    print(str_content[0:6])

<class 'bytes'>
b'\x00\x00\x00'
000000


In [7]:
def find_nalu(h264_data):
    nalus = []
    i = 0
    while i < len(h264_data) - 6:
        # find nalu start point based on start code prefix
        if h264_data[i:i+2] == '00' and h264_data[i+2:i+4] == '00' and h264_data[i+4:i+6] == '01':
            start = i + 6
            i += 6
            while i < len(h264_data) - 6 and not (h264_data[i:i+2] == '00' and h264_data[i+2:i+4] == '00' and h264_data[i+4:i+6] == '01'):
                i += 2
            nalus.append(h264_data[start:i])
        else:
            i += 2
    return nalus

In [8]:
nalu_list = find_nalu(str_content)
print(len(nalu_list))
for i in range(10):
    print(nalu_list[i])

305
6764000dacd941609684000003000400000300ca3c50a65800
68ebe3cb22c0
...


In [9]:
def hex_to_bin(hex_string):
    # 初始化二进制字符串
    bin_string = ''
    # 遍历十六进制字符串的每个字符
    for char in hex_string:
        # 将十六进制字符转换为十进制，然后转换为二进制并去掉前缀'0b'
        bin_char = bin(int(char, 16))[2:]
        # 为转换后的二进制字符串补足4位
        bin_char = bin_char.zfill(4)
        # 将处理后的二进制字符串拼接
        bin_string += bin_char
    return bin_string

# # 示例：将十六进制字符'1A3'转换为二进制
# hex_string = '1A3'
# bin_string = hex_to_bin(hex_string)
# print(f"十六进制 '{hex_string}' 转换为二进制是 '{bin_string}'")

In [26]:
class NAL_unit:
    def __init__(self, nalu):
        self.nalu = nalu
        self.forbidden_zero_bit = 0
        self.nal_ref_idc = 0
        self.nal_unit_type = -1
        self.rbsp_byte = ''
        

    def nal_unit(self):
        # print(len(nalu))
        NumBytesInNALunit = int(len(self.nalu) / 2)
        binary_string = hex_to_bin(self.nalu)
        # print(binary_string[:8])
        self.forbidden_zero_bit, binary_string = Decode_util.fn(binary_string, 1)
        self.nal_ref_idc, binary_string = Decode_util.uv(binary_string, 2)
        # print(binary_string[:5])
        self.nal_unit_type, binary_string = Decode_util.uv(binary_string, 5)
        # print(nal_unit_type)

        NumBytesInRBSP = 0
        nalUnitHeaderBytes = 1


        if self.nal_unit_type == 14 or self.nal_unit_type == 20 or self.nal_unit_type == 21:
            if self.nal_unit_type != 21:
                svc_extension_flag, binary_string = Decode_util.uv(binary_string, 1)
            else:
                avc_3d_extension_flag,binary_string = Decode_util.uv(binary_string, 1)
            print("count")
        # Need to be continued

        for i in range(nalUnitHeaderBytes, NumBytesInNALunit):
            if i + 2 < NumBytesInNALunit and binary_string[:24] == hex_to_bin('000003'):
                b8value, binary_string = Decode_util.b8(binary_string)
                self.rbsp_byte = self.rbsp_byte + b8value
                b8value, binary_string = Decode_util.b8(binary_string)
                self.rbsp_byte = self.rbsp_byte + b8value
                b8value, binary_string = Decode_util.b8(binary_string)
                i += 2
                print('delete emulation_prevention_three_byte')
            else:
                b8value, binary_string = Decode_util.b8(binary_string)
                self.rbsp_byte = self.rbsp_byte + b8value


In [ ]:
def seq_parameter_set_data(rbsp_byte):
    profile_idc, rbsp_byte = Decode_util.uv(rbsp_byte, 8)
    constraint_set0_flag, rbsp_byte = Decode_util.uv(rbsp_byte, 1)
    constraint_set1_flag, rbsp_byte = Decode_util.uv(rbsp_byte, 1)
    constraint_set2_flag, rbsp_byte = Decode_util.uv(rbsp_byte, 1)
    constraint_set3_flag, rbsp_byte = Decode_util.uv(rbsp_byte, 1)
    constraint_set4_flag, rbsp_byte = Decode_util.uv(rbsp_byte, 1)
    constraint_set5_flag, rbsp_byte = Decode_util.uv(rbsp_byte, 1)
    reserved_zero_2bits, rbsp_byte = Decode_util.uv(rbsp_byte, 2)
    level_idc, rbsp_byte = Decode_util.uv(rbsp_byte, 8)
    seq_parameter_set_id, rbsp_byte = Decode_util.uev(rbsp_byte)

    if profile_idc == 100 or profile_idc == 110 or profile_idc == 122 or profile_idc == 244 or profile_idc == 44 or
    profile_idc == 83 or profile_idc == 86 or profile_idc == 118 or profile_idc == 128 or profile_idc == 138 or profile_idc == 139 or
    profile_idc == 134 or profile_idc == 135:
        chroma_format_idc, rbsp_byte = Decode_util.uev(rbsp_byte)
        if chroma_format_idc == 3:
            separate_colour_plane_flag, rbsp_byte = Decode_util.uv(rbsp_byte, 1)
        bit_depth_luma_minus8, rbsp_byte = Decode_util.uev(rbsp_byte)
        bit_depth_chroma_minus8, rbsp_byte = Decode_util.uev(rbsp_byte)
        qpprime_y_zero_transform_bypass_flag, rbsp_byte = Decode_util.uv(rbsp_byte, 1)
        seq_scaling_matrix_present_flag, rbsp_byte = Decode_util.uv(rbsp_byte, 1)
        if seq_scaling_matrix_present_flag:
            seq_scaling_list_present_flag = []
            for i in range(8 if chroma_format_idc != 3 else 12):
                value, rbsp_byte = Decode_util.uv(rbsp_byte, 1)
                seq_scaling_list_present_flag.append(value)
                #need to be contiuned

    log2_max_frame_num_minus4, rbsp_byte = Decode_util.uev(rbsp_byte)
    pic_order_cnt_type, rbsp_byte = Decode_util.uev(rbsp_byte)
    if pic_order_cnt_type == 0:
        log2_max_pic_order_cnt_lsb_minus4, rbsp_byte = Decode_util.uev(rbsp_byte)
    elif pic_order_cnt_type == 1:
        delta_pic_order_always_zero_flag, rbsp_byte = Decode_util.uv(rbsp_byte, 1)
        offset_for_non_ref_pic, rbsp_byte = Decode_util.sev(rbsp_byte)
        offset_for_top_to_bottom_field, rbsp_byte = Decode_util.sev(rbsp_byte)
        num_ref_frames_in_pic_order_cnt_cycle, rbsp_byte = Decode_util.uev(rbsp_byte)
        offset_for_ref_frame = []
        for i in range(num_ref_frames_in_pic_order_cnt_cycle):
            value, rbsp_byte = Decode_util.sev(rbsp_byte)
            offset_for_ref_frame.append(value)
    max_num_ref_frames, rbsp_byte = Decode_util.uev(rbsp_byte)
    gaps_in_frame_num_value_allowed_flag, rbsp_byte = Decode_util.uv(rbsp_byte, 1)
    pic_width_in_mbs_minus1, rbsp_byte = Decode_util.uev(rbsp_byte)
    pic_height_in_map_units_minus1, rbsp_byte = Decode_util.uev(rbsp_byte)
    frame_mbs_only_flag, rbsp_byte = Decode_util.uv(rbsp_byte, 1)
    if not frame_mbs_only_flag:
        mb_adaptive_frame_field_flag, rbsp_byte = Decode_util.uv(rbsp_byte, 1)
    direct_8x8_inference_flag, rbsp_byte = Decode_util.uv(rbsp_byte, 1)
    frame_cropping_flag, rbsp_byte = Decode_util.uv(rbsp_byte, 1)
    if frame_cropping_flag:
        frame_crop_left_offset, rbsp_byte = Decode_util.uev(rbsp_byte)
        frame_crop_right_offset, rbsp_byte = Decode_util.uev(rbsp_byte)
        frame_crop_top_offset, rbsp_byte = Decode_util.uev(rbsp_byte)
        frame_crop_bottom_offset, rbsp_byte = Decode_util.uev(rbsp_byte)
    vui_parameters_present_flag, rbsp_byte = Decode_util.uv(rbsp_byte, 1)
    # if vui_parameters_present_flag:
    #     vui_parameters()


def seq_parameter_set_rbsp(rbsp_byte):
    seq_parameter_set_data(rbsp_byte)
    # rbsp_trailing_bits()

In [29]:
x = 13
for i in range(2 if x <11 else 3):
    print(i)

0
1
2


In [25]:
for nalu in nalu_list:
    nal_unit_obj = NAL_unit(nalu)
    nal_unit_obj.nal_unit()
    print(nal_unit_obj.nal_unit_type)

delete emulation_prevention_three_byte
delete emulation_prevention_three_byte
7
8
6
5
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
delete emulation_prevention_three_byte
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
delete emulation_prevention_three_byte
delete emulation_prevention_three_byte
7
8
5
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [2]:
bitstream = '001000110001010'
Decode_util.b8(bitstream)

('00100011', '0001010')